In [2]:
import requests
from urllib import parse

In [3]:
#오픈 api 소스 파이썬으로 불러오기
url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson"
api_key_utf8 = "개인 인코딩 key"
api_key_decode = requests.utils.unquote(api_key_utf8, encoding = "utf-8")

params={
    "ServiceKey": "개인 인코딩 key",
    "startCreateDt": 20200221,
    "endCreateDt": 20200221
}

response = requests.get(url, params = params)

In [4]:
print(response.text)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><accDefRate>1.340666896</accDefRate><accExamCnt>14816</accExamCnt><accExamCompCnt>11636</accExamCompCnt><careCnt>139</careCnt><clearCnt>16</clearCnt><createDt>2020-02-21 16:00:00.000</createDt><deathCnt>1</deathCnt><decideCnt>156</decideCnt><examCnt>3180</examCnt><resutlNegCnt>11480</resutlNegCnt><seq>33</seq><stateDt>20200221</stateDt><stateTime>16:00</stateTime><updateDt>2021-10-07 10:30:51.51</updateDt></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>1</totalCount></body></response>


In [8]:
import requests
import pandas as pd
from typing import Union
from bs4 import BeautifulSoup
from datetime import date, datetime

In [44]:
def getCovid19Info(start_date: Union[date, datetime], end_date: Union[date, datetime]):
    url = "http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson"
    api_key_utf8 = "개인 인코딩 key"
    api_key_decode = requests.utils.unquote(api_key_utf8, encoding = "utf-8")

    params={
        "ServiceKey": "개인 인코딩 key",
        "startCreateDt": int("{:04d}{:02d}{:02d}".format(start_date.year, start_date.month, start_date.day)),
        "endCreateDt": int("{:04d}{:02d}{:02d}".format(end_date.year, end_date.month, end_date.day)),
}

    response = requests.get(url, params = params)
    elapsed_us  = response.elapsed.microseconds
    print("Request Done, Elapsed: {} seconds". format(elapsed_us / 1e6))
    return BeautifulSoup(response.text,"lxml")

def getCovid19DataFrame(start_date: Union[date, datetime], end_date: Union[date, datetime]) -> pd.DataFrame:
    convert_method = {
        "accdefrate" : float,
        "accexamcnt" : int,
        "accexamcompcnt" : int,
        "carecnt": int,
        "clearcnt": int,
        "createdt": lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f"),
        "deathcnt": int,
        "decidecnt": int,
        "examcnt": int,
        "resutlnegcnt": int,
        "seq": int,
        "statedt": lambda x: datetime.strptime(x, "%Y%m%d"),
        "statetime": str,
        "updatedt": lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
        
    }
    
    temp = getCovid19Info(start_date, end_date)
    items = temp.find("items")
    item_list = []
    for item in items:
        item_dict = {}
        for tag in list(item):
            try:
                item_dict[tag.name] = convert_method[tag.name](tag.text)
            except Exception:
                item_dict[tag.name] = None
        item_list.append(item_dict)
    df = pd.DataFrame(item_list)
    return df

def getAllCovid19Data() -> pd.DataFrame:
    now = datetime.now()
    df = getCovid19DataFrame(date(2019,1,1), now)
    print("Today: {}\nLoaded {} Records".format(now.strftime("%Y-%m-&d"), df.shape[0]))
    return df

In [45]:
df = getAllCovid19Data()

Request Done, Elapsed: 0.202465 seconds
Today: 2021-11-&d
Loaded 666 Records


In [47]:
df[:]

,accdefrate,accexamcnt,accexamcompcnt,carecnt,clearcnt,createdt,deathcnt,decidecnt,examcnt,resutlnegcnt,seq,statedt,statetime,updatedt
0,2.680373,16525389,14907924,31952,364499,2021-11-16 08:39:32.225,3137,399588,1617465,14508336,699,2021-11-16,00:00,None
1,2.679075,16470972,14835866,31515,362834,2021-11-15 09:35:56.024,3115,397464,1635106,14438402,698,2021-11-15,00:00,None
2,2.667307,16440429,14826151,31465,360891,2021-11-14 09:44:03.248,3103,395459,1614278,14430692,697,2021-11-14,00:00,None
3,2.652760,16408600,13689934,304050,359553,2021-11-13 09:40:09.109,3083,393041,1592295,14423264,696,2021-11-13,00:00,None
4,2.642399,16361030,14786451,31051,356615,2021-11-12 09:27:05.279,3051,390717,1574579,14395734,695,2021-11-12,00:00,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661,6.896552,29,29,2,0,2020-01-24 00:00:00.000,0,2,0,27,5,2020-01-24,00:00,None
662,4.000000,25,25,1,0,2020-01-23 09:00:00.000,0,1,0,24,4,2020-01-23,09:00,None
663,4.761905,21,21,1,0,2020-01-22 12:00:00.000,0,1,0,20,3,2020-01-22,12:00,None
664,6.666667,15,15,1,0,2020-01-21 09:00:00.000,0,1,0,14,2,2020-01-21,09:00,None


In [49]:
df.to_csv("C:\hwang\corona") # csv파일로 데이터 프레임 저장

In [55]:
df.to_excel("C:\hwang\corona.xlsx") # 엑셀 파일로 데이터 프레임 저장